# Predicting if a customer leaves or stays with a bank using ANN

Importing the libraries numpy, matplotlib and pandas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Importing the dataset containing the Bank Users Details

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Segregating the Dependent data from the Independent data in the imported dataset.

The Row Numer, CustomerId and Surname are ignored for this analysis.

In [3]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

The imported data contains categorical data (Geograpgy and Gender) which need to be numerical to perform neural networks on the data.


The following procedure is used to convert categorical data into numerical data without changing the original meaning of the data.


The classes LabelEncoder and OneHotEncoder from Scikit Learn are used for this purporse.

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Label encoder for Geography

In [5]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

Label Encoder for Gender

In [6]:
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [7]:
X[:8, :4] #The categorical field Geography had three countries which are turned into 0s and 1s

array([[619, 0, 0, 42],
       [608, 2, 0, 41],
       [502, 0, 0, 42],
       [699, 0, 0, 39],
       [850, 2, 0, 43],
       [645, 2, 1, 44],
       [822, 0, 1, 50],
       [376, 1, 0, 29]], dtype=object)

In [8]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

Splitting the imported data into Test (20 % of data) and Training Data (80% of data).

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Since the numerical data is quite varied, the data is scaled so that the range of data across all the fields is reduced.

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Building the Artificial Neural Network Model

Importing Keras library and also a few other modules from keras

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


Initializing the Artificial Neural Network

In [29]:
classifier = Sequential()

Adding the Input Layer and the First Hidden layer 

In [30]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

Adding the Second Hidden layer

In [31]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

Adding the Output Layer

In [32]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

Compiling the Artificial Neural Network

In [33]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Fitting the Artificial Neural Network to the Trainig set created above

In [34]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
8000/8000 [==============================] - 7s 864us/step - loss: 0.4825 - acc: 0.7957
Epoch 2/10
8000/8000 [==============================] - 3s 385us/step - loss: 0.4273 - acc: 0.7960
Epoch 3/10
8000/8000 [==============================] - 3s 328us/step - loss: 0.4219 - acc: 0.7972
Epoch 4/10
8000/8000 [==============================] - 3s 330us/step - loss: 0.4183 - acc: 0.8207
Epoch 5/10
8000/8000 [==============================] - 3s 336us/step - loss: 0.4158 - acc: 0.8250
Epoch 6/10
8000/8000 [==============================] - 3s 342us/step - loss: 0.4144 - acc: 0.8307
Epoch 7/10
8000/8000 [==============================] - 2s 279us/step - loss: 0.4130 - acc: 0.8291
Epoch 8/10
8000/8000 [==============================] - 2s 270us/step - loss: 0.4122 - acc: 0.8309
Epoch 9/10
8000/8000 [==============================] - 2s 275us/step - loss: 0.4107 - acc: 0.8336
Epoch 10/10
8000/8000 [==============================] - 2s 277us/step - loss: 0.4095 - acc: 0.8337


Predicting the Test Results

In [18]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

Calculating the confusion matrix to interpret the results clearly

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1557,   38],
       [ 282,  123]], dtype=int64)

In [20]:
Accuracy = (1557 + 123)/(1557 + 38 + 282 + 123)
Accuracy

0.84

Evaluating the Artificial Neural Network using K-Fold Cross Validation Technique

In [41]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [42]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()

Tuning the Artificial Neural Network using Grid Search Technique

In [21]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [22]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)

In [23]:
parameters = {'batch_size': [25, 32],
              'epochs': [10, 50],
              'optimizer': ['adam', 'rmsprop']}

In [24]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

In [25]:
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/10
7200/7200 [==============================] - 1s 119us/step - loss: 0.5641 - acc: 0.7964
Epoch 2/10
7200/7200 [==============================] - 1s 76us/step - loss: 0.4365 - acc: 0.7971
Epoch 3/10
7200/7200 [==============================] - 1s 78us/step - loss: 0.4303 - acc: 0.7971
Epoch 4/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.4267 - acc: 0.7971
Epoch 5/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.4245 - acc: 0.7971
Epoch 6/10
7200/7200 [==============================] - 1s 73us/step - loss: 0.4223 - acc: 0.7971
Epoch 7/10
7200/7200 [==============================] - 1s 72us/step - loss: 0.4203 - acc: 0.8069
Epoch 8/10
7200/7200 [==============================] - 1s 74us/step - loss: 0.4189 - acc: 0.8200
Epoch 9/10
7200/7200 [==============================] - 1s 77us/step - loss: 0.4174 - acc: 0.8242
Epoch 10/10
7200/7200 [==============================] - 1s 79us/step - loss: 0.4164 - acc: 0.8261
Epoch 1/10
7200/72

7200/7200 [==============================] - 1s 71us/step - loss: 0.5391 - acc: 0.7957
Epoch 5/10
7200/7200 [==============================] - 0s 69us/step - loss: 0.5245 - acc: 0.7957
Epoch 6/10
7200/7200 [==============================] - 0s 62us/step - loss: 0.5159 - acc: 0.7957
Epoch 7/10
7200/7200 [==============================] - 1s 72us/step - loss: 0.5111 - acc: 0.7957
Epoch 8/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.5086 - acc: 0.7957
Epoch 9/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.5074 - acc: 0.7957
Epoch 10/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.5068 - acc: 0.7957
Epoch 1/10
7200/7200 [==============================] - 1s 191us/step - loss: 0.5534 - acc: 0.7958
Epoch 2/10
7200/7200 [==============================] - 0s 68us/step - loss: 0.4372 - acc: 0.7961
Epoch 3/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.4307 - acc: 0.7961
Epoch 4/10
7200/7200 [=======

7200/7200 [==============================] - 1s 74us/step - loss: 0.4009 - acc: 0.8283
Epoch 8/10
7200/7200 [==============================] - 1s 72us/step - loss: 0.3946 - acc: 0.8293
Epoch 9/10
7200/7200 [==============================] - 1s 70us/step - loss: 0.3887 - acc: 0.8286
Epoch 10/10
7200/7200 [==============================] - 1s 70us/step - loss: 0.3835 - acc: 0.8329
Epoch 1/10
7200/7200 [==============================] - 2s 218us/step - loss: 0.5715 - acc: 0.7951
Epoch 2/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.4393 - acc: 0.7963
Epoch 3/10
7200/7200 [==============================] - 1s 72us/step - loss: 0.4307 - acc: 0.7962
Epoch 4/10
7200/7200 [==============================] - 1s 77us/step - loss: 0.4267 - acc: 0.7962
Epoch 5/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.4233 - acc: 0.7963
Epoch 6/10
7200/7200 [==============================] - 1s 94us/step - loss: 0.4202 - acc: 0.7972
Epoch 7/10
7200/7200 [=======

7200/7200 [==============================] - 0s 67us/step - loss: 0.3961 - acc: 0.8358
Epoch 50/50
7200/7200 [==============================] - 0s 67us/step - loss: 0.3963 - acc: 0.8364
Epoch 1/50
7200/7200 [==============================] - 2s 214us/step - loss: 0.5487 - acc: 0.7950
Epoch 2/50
7200/7200 [==============================] - 0s 65us/step - loss: 0.4332 - acc: 0.7967
Epoch 3/50
7200/7200 [==============================] - 0s 66us/step - loss: 0.4271 - acc: 0.7967
Epoch 4/50
7200/7200 [==============================] - 0s 66us/step - loss: 0.4232 - acc: 0.7967
Epoch 5/50
7200/7200 [==============================] - 0s 65us/step - loss: 0.4203 - acc: 0.7967
Epoch 6/50
7200/7200 [==============================] - 1s 74us/step - loss: 0.4180 - acc: 0.8142
Epoch 7/50
7200/7200 [==============================] - 1s 90us/step - loss: 0.4166 - acc: 0.8233
Epoch 8/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.4154 - acc: 0.8243
Epoch 9/50
7200/7200 [=======

7200/7200 [==============================] - 0s 69us/step - loss: 0.4056 - acc: 0.8349
Epoch 32/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4052 - acc: 0.8360
Epoch 33/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4049 - acc: 0.8346
Epoch 34/50
7200/7200 [==============================] - 1s 73us/step - loss: 0.4044 - acc: 0.8357
Epoch 35/50
7200/7200 [==============================] - 1s 86us/step - loss: 0.4043 - acc: 0.8356
Epoch 36/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4043 - acc: 0.8364
Epoch 37/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.4040 - acc: 0.8349
Epoch 38/50
7200/7200 [==============================] - 1s 79us/step - loss: 0.4035 - acc: 0.8365
Epoch 39/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.4040 - acc: 0.8353
Epoch 40/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.4031 - acc: 0.8361: 0s - loss: 0.3861 - 

...

7200/7200 [==============================] - 1s 124us/step - loss: 0.4137 - acc: 0.8251
Epoch 9/50
7200/7200 [==============================] - 1s 111us/step - loss: 0.4112 - acc: 0.8281
Epoch 10/50
7200/7200 [==============================] - 1s 107us/step - loss: 0.4090 - acc: 0.8297
Epoch 11/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4075 - acc: 0.8301
Epoch 12/50
7200/7200 [==============================] - 1s 75us/step - loss: 0.4058 - acc: 0.8326
Epoch 13/50
7200/7200 [==============================] - 0s 66us/step - loss: 0.4046 - acc: 0.8343
Epoch 14/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.4031 - acc: 0.8349
Epoch 15/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.4021 - acc: 0.8356
Epoch 16/50
7200/7200 [==============================] - 0s 64us/step - loss: 0.4011 - acc: 0.8363
Epoch 17/50
7200/7200 [==============================] - 0s 64us/step - loss: 0.4002 - acc: 0.8339
Epoch 18/50
7200/720

7200/7200 [==============================] - 1s 74us/step - loss: 0.4067 - acc: 0.8343
Epoch 41/50
7200/7200 [==============================] - 1s 73us/step - loss: 0.4065 - acc: 0.8335
Epoch 42/50
7200/7200 [==============================] - 1s 72us/step - loss: 0.4064 - acc: 0.8331
Epoch 43/50
7200/7200 [==============================] - 1s 72us/step - loss: 0.4058 - acc: 0.8336
Epoch 44/50
7200/7200 [==============================] - 1s 71us/step - loss: 0.4060 - acc: 0.8325
Epoch 45/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.4059 - acc: 0.8329
Epoch 46/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.4056 - acc: 0.8342
Epoch 47/50
7200/7200 [==============================] - 1s 110us/step - loss: 0.4055 - acc: 0.8331
Epoch 48/50
7200/7200 [==============================] - 1s 126us/step - loss: 0.4055 - acc: 0.8346
Epoch 49/50
7200/7200 [==============================] - 1s 139us/step - loss: 0.4051 - acc: 0.8342
Epoch 50/50
7200/72

7200/7200 [==============================] - 1s 143us/step - loss: 0.4010 - acc: 0.8353
Epoch 23/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4002 - acc: 0.8360
Epoch 24/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.3998 - acc: 0.8344
Epoch 25/50
7200/7200 [==============================] - 1s 75us/step - loss: 0.3995 - acc: 0.8351
Epoch 26/50
7200/7200 [==============================] - 1s 79us/step - loss: 0.3990 - acc: 0.8346
Epoch 27/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.3985 - acc: 0.8347
Epoch 28/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3983 - acc: 0.8354
Epoch 29/50
7200/7200 [==============================] - 1s 84us/step - loss: 0.3981 - acc: 0.8361
Epoch 30/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3977 - acc: 0.8357
Epoch 31/50
7200/7200 [==============================] - 1s 72us/step - loss: 0.3974 - acc: 0.8353
Epoch 32/50
7200/720

7200/7200 [==============================] - 1s 95us/step - loss: 0.4276 - acc: 0.7957
Epoch 5/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.4243 - acc: 0.7957
Epoch 6/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4219 - acc: 0.7957
Epoch 7/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4195 - acc: 0.7957
Epoch 8/50
7200/7200 [==============================] - 1s 127us/step - loss: 0.4176 - acc: 0.8047
Epoch 9/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4161 - acc: 0.8240
Epoch 10/50
7200/7200 [==============================] - 1s 141us/step - loss: 0.4146 - acc: 0.8271
Epoch 11/50
7200/7200 [==============================] - 1s 120us/step - loss: 0.4134 - acc: 0.8278
Epoch 12/50
7200/7200 [==============================] - 1s 109us/step - loss: 0.4123 - acc: 0.8282
Epoch 13/50
7200/7200 [==============================] - 1s 110us/step - loss: 0.4112 - acc: 0.8307
Epoch 14/50
7200/7200

7200/7200 [==============================] - 1s 77us/step - loss: 0.4062 - acc: 0.8333
Epoch 37/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4060 - acc: 0.8336
Epoch 38/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4057 - acc: 0.8332
Epoch 39/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.4056 - acc: 0.8342
Epoch 40/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4052 - acc: 0.8336
Epoch 41/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4054 - acc: 0.8344
Epoch 42/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4051 - acc: 0.8336
Epoch 43/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4047 - acc: 0.8340
Epoch 44/50
7200/7200 [==============================] - 1s 133us/step - loss: 0.4047 - acc: 0.8346
Epoch 45/50
7200/7200 [==============================] - 1s 110us/step - loss: 0.4046 - acc: 0.8351
Epoch 46/50
7200/7

In [26]:
best_parameters

{'batch_size': 32, 'epochs': 50, 'optimizer': 'rmsprop'}

In [27]:
best_accuracy

0.83950000000000002

Predict if the customer with the following information will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40

Tenure: 3

Balance: 60000

Number of Products: 2

Has Credit Card: Yes

Is Active Member: Yes

Estimated Salary: 50000

In [36]:
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]], dtype=bool)

The value False implies that the customer stays with the bank.